In [1]:
from func_def import *

In [2]:
df = pd.read_csv('data_pre_processed.csv',index_col=[0]).reset_index(drop=True)
df.tail(3)

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head_perc,f_sig_str_body_perc,f_sig_str_leg_perc,f_sig_str_dist_perc,f_sig_str_clinch_perc,f_sig_str_ground_perc,o_sig_str_head_perc,o_sig_str_body_perc,o_sig_str_leg_perc,o_sig_str_dist_perc,o_sig_str_clinch_perc,o_sig_str_ground_perc
7192,http://www.ufcstats.com/fight-details/ccee020b...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Johnny Rhode,David Levick,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,9of15,1of1,1of1,1of1,1of1,9of15,4of5,0of0,0of0,1of2,2of2,1of1,0.81,0.09,0.09,0.09,0.09,0.81,1.0,0.0,0.0,0.25,0.5,0.25
7193,http://www.ufcstats.com/fight-details/4b9ae533...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Patrick Smit,Ray Wizar,Open Weight Bout,Submission,1,0:58,No Time Limit,0of0,1of1,0of0,0of0,1of1,0of0,0of0,0of0,1of1,1of1,0of0,0of0,0.00,1.00,0.00,0.00,1.00,0.00,0.0,0.0,1.0,1.00,0.0,0.00
7194,http://www.ufcstats.com/fight-details/4acab678...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Scott Morri,Sean Daughert,Open Weight Bout,Submission,1,0:20,No Time Limit,1of1,0of0,0of0,0of0,1of1,0of0,0of2,0of0,0of2,0of3,0of1,0of0,1.00,0.00,0.00,0.00,1.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00


In [14]:
# data enrichments
NEW_DATA_COLUMNS = ['win_perc']
for column in NEW_DATA_COLUMNS:
    for prefix in ['f_','o_']:
        df[f'{prefix}{column}'] = [get_win_perc(fighter=df.loc[i,'fighter'],time=df.loc[i,'date'],data=df) for i in range(len(df))]

df.tail()

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head_perc,f_sig_str_body_perc,f_sig_str_leg_perc,f_sig_str_dist_perc,f_sig_str_clinch_perc,f_sig_str_ground_perc,o_sig_str_head_perc,o_sig_str_body_perc,o_sig_str_leg_perc,o_sig_str_dist_perc,o_sig_str_clinch_perc,o_sig_str_ground_perc,f_win_perc,o_win_perc
7190,http://www.ufcstats.com/fight-details/3b020d49...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Orlando Wiet,Robert Lucarell,Open Weight Bout,KO/TKO,1,2:50,No Time Limit,7of11,1of1,0of0,1of3,0of0,7of9,1of2,0of1,1of3,2of6,0of0,0of0,0.87,0.12,0.00,0.12,0.00,0.87,0.5,0.0,0.5,1.00,0.0,0.00,0.0,0.0
7191,http://www.ufcstats.com/fight-details/d917c8c7...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Frank Hamake,Thaddeus Luste,Open Weight Bout,Submission,1,4:52,No Time Limit,2of3,0of0,0of0,1of1,0of0,1of2,0of0,0of0,0of0,0of0,0of0,0of0,1.00,0.00,0.00,0.50,0.00,0.50,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0
7192,http://www.ufcstats.com/fight-details/ccee020b...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Johnny Rhode,David Levick,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,9of15,1of1,1of1,1of1,1of1,9of15,4of5,0of0,0of0,1of2,2of2,1of1,0.81,0.09,0.09,0.09,0.09,0.81,1.0,0.0,0.0,0.25,0.5,0.25,0.0,0.0
7193,http://www.ufcstats.com/fight-details/4b9ae533...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Patrick Smit,Ray Wizar,Open Weight Bout,Submission,1,0:58,No Time Limit,0of0,1of1,0of0,0of0,1of1,0of0,0of0,0of0,1of1,1of1,0of0,0of0,0.00,1.00,0.00,0.00,1.00,0.00,0.0,0.0,1.0,1.00,0.0,0.00,0.0,0.0
7194,http://www.ufcstats.com/fight-details/4acab678...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Scott Morri,Sean Daughert,Open Weight Bout,Submission,1,0:20,No Time Limit,1of1,0of0,0of0,0of0,1of1,0of0,0of2,0of0,0of2,0of3,0of1,0of0,1.00,0.00,0.00,0.00,1.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0


In [3]:
#tester unit
from func_def import *
df = pd.read_csv('data_pre_processed.csv',index_col=[0]).reset_index(drop=True)
NAME = 'Max Holloway'
df[(df.fighter==NAME)|(df.opponent==NAME)]

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head_perc,f_sig_str_body_perc,f_sig_str_leg_perc,f_sig_str_dist_perc,f_sig_str_clinch_perc,f_sig_str_ground_perc,o_sig_str_head_perc,o_sig_str_body_perc,o_sig_str_leg_perc,o_sig_str_dist_perc,o_sig_str_clinch_perc,o_sig_str_ground_perc
45,http://www.ufcstats.com/fight-details/96704d7b...,http://www.ufcstats.com/event-details/89a40703...,2023-08-26,W,Max Holloway,Chan Sung Jung,Featherweight Bout,KO/TKO,3,0:23,5 Rnd (5-5-5-5-5),34of77,31of35,10of16,71of124,1of1,3of3,25of110,6of8,3of6,34of122,0of2,0of0,0.45,0.41,0.13,0.94,0.01,0.04,0.73,0.17,0.08,1.00,0.00,0.00
265,http://www.ufcstats.com/fight-details/6049301b...,http://www.ufcstats.com/event-details/15b1b21c...,2023-04-15,W,Max Holloway,Arnold Allen,Featherweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),77of180,42of48,28of34,146of261,1of1,0of0,48of173,18of36,10of16,76of225,0of0,0of0,0.52,0.28,0.19,0.99,0.00,0.00,0.63,0.23,0.13,1.00,0.00,0.00
658,http://www.ufcstats.com/fight-details/4047e981...,http://www.ufcstats.com/event-details/4a9e3056...,2022-07-02,W,Alexander Volkanovski,Max Holloway,UFC Featherweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),152of316,19of24,28of32,175of340,24of32,0of0,64of213,42of59,21of29,109of283,18of18,0of0,0.76,0.09,0.14,0.87,0.12,0.00,0.50,0.33,0.16,0.85,0.14,0.00
965,http://www.ufcstats.com/fight-details/147e984a...,http://www.ufcstats.com/event-details/b5abaa65...,2021-11-13,W,Max Holloway,Yair Rodriguez,Featherweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),129of278,66of83,35of43,187of347,22of24,21of33,78of247,24of41,57of64,152of343,5of5,2of4,0.56,0.28,0.15,0.81,0.09,0.09,0.49,0.15,0.35,0.95,0.03,0.01
1401,http://www.ufcstats.com/fight-details/a4817b7e...,http://www.ufcstats.com/event-details/307064d3...,2021-01-16,W,Max Holloway,Calvin Katta,Featherweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),274of553,117of133,54of58,439of737,6of7,0of0,99of237,16of23,18of23,129of279,4of4,0of0,0.61,0.26,0.12,0.98,0.01,0.00,0.74,0.12,0.13,0.96,0.03,0.00
1662,http://www.ufcstats.com/fight-details/94b1596c...,http://www.ufcstats.com/event-details/dbd198f7...,2020-07-11,W,Alexander Volkanovski,Max Holloway,UFC Featherweight Title Bout,Decision - Split,5,5:00,5 Rnd (5-5-5-5-5),64of161,6of10,67of101,136of265,1of6,0of1,44of167,27of46,31of55,95of258,7of10,0of0,0.46,0.04,0.48,0.99,0.00,0.00,0.43,0.26,0.30,0.93,0.06,0.00
1870,http://www.ufcstats.com/fight-details/efc40ec4...,http://www.ufcstats.com/event-details/4565d435...,2019-12-14,L,Max Holloway,Alexander Volkanovski,UFC Featherweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),85of233,32of44,17of26,132of301,2of2,0of0,67of178,15of32,75of93,156of302,1of1,0of0,0.63,0.23,0.12,0.98,0.01,0.00,0.42,0.09,0.47,0.99,0.00,0.00
2078,http://www.ufcstats.com/fight-details/c4704eab...,http://www.ufcstats.com/event-details/abcf7e55...,2019-07-27,W,Max Holloway,Frankie Edgar,UFC Featherweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),109of295,20of33,0of0,120of319,8of8,1of1,57of195,21of48,18of21,93of252,2of10,1of2,0.84,0.15,0.00,0.93,0.06,0.00,0.59,0.21,0.18,0.96,0.02,0.01
2235,http://www.ufcstats.com/fight-details/87126427...,http://www.ufcstats.com/event-details/b0550072...,2019-04-13,L,Max Holloway,Dustin Poirier,UFC Interim Lightweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),141of394,39of50,1of2,167of425,14of21,0of0,153of349,18of24,7of8,165of365,13of16,0of0,0.77,0.21,0.00,0.92,0.07,0.00,0.85,0.10,0.03,0.92,0.07,0.00
2397,http://www.ufcstats.com/fight-details/4e55251b...,http://www.ufcstats.com/event-details/2a74bbba...,2018-12-08,W,Max Holloway,Brian Ortega,UFC Featherweight Title Bout,TKO - Doctor's Stoppage,4,5:00,5 Rnd (5-5-5-5-5),244of430,41of55,5of5,281of479,8

In [4]:
#test
get_win_streak(fighter=NAME,time='2024-08-26',data=df)
#get_last_n_stat_mean(fighter=NAME,stat_col='sig_str_head_perc',time='2023-08-26',data=df,n=5)

0

In [51]:
# Pick features
FEATURES = ['distance_sig_str','clinch_sig_str','ground_sig_str']
#FEATURES = ['head_sig_str','body_sig_str','leg_sig_str','clinch_sig_str','ground_sig_str']

feature_columns = [f'f_{item}' for item in FEATURES] + [f'o_{item}' for item in FEATURES]
print(feature_columns)

['f_distance_sig_str', 'f_clinch_sig_str', 'f_ground_sig_str', 'o_distance_sig_str', 'o_clinch_sig_str', 'o_ground_sig_str']


In [52]:
data = df[['date','fighter','opponent','result'] + feature_columns].copy()

for prefix in ['f_','o_']:
    for feature in FEATURES:
        data[f'{prefix}{feature}'] = pd.to_numeric([int(item.split('of')[0]) for item in df[f'{prefix}{feature}']])
        data[f'{prefix}{feature}_att'] = pd.to_numeric([int(item.split('of')[1]) for item in df[f'{prefix}{feature}']])

data.tail(3)

,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att
7192,1994-03-11,Johnny Rhode,David Levick,W,1,1,9,1,2,1,1,1,15,2,2,1
7193,1994-03-11,Patrick Smit,Ray Wizar,W,0,1,0,1,0,0,0,1,0,1,0,0
7194,1994-03-11,Scott Morri,Sean Daughert,W,0,1,0,0,0,0,0,1,0,3,1,0


In [53]:
N = 3 #recent fights to include for certain stats
STATS = [item for item in FEATURES] + [f'{item}_att' for item in FEATURES]
for stat in STATS:
    for index in tqdm(data.index):
        data.loc[index,f'f_{stat}_mean'] = get_mean_stat(fighter=data.loc[index,'fighter'],stat_col=f'f_{stat}',time=data.loc[index,'date'],data=data)
        data.loc[index,f'o_{stat}_mean'] = get_mean_stat(fighter=data.loc[index,'opponent'],stat_col=f'o_{stat}',time=data.loc[index,'date'],data=data)
        data.loc[index,f'f_{stat}_last_{N}_mean'] = get_last_n_stat_mean(fighter=data.loc[index,'fighter'],stat_col=f'f_{stat}',time=data.loc[index,'date'],data=data,n=N)
        data.loc[index,f'o_{stat}_last_{N}_mean'] = get_last_n_stat_mean(fighter=data.loc[index,'opponent'],stat_col=f'o_{stat}',time=data.loc[index,'date'],data=data,n=N)

data.tail(3)

100%|██████████| 7195/7195 [02:24<00:00, 49.75it/s]


,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_distance_sig_str_mean,o_distance_sig_str_mean,f_distance_sig_str_last_3_mean,o_distance_sig_str_last_3_mean,f_clinch_sig_str_mean,o_clinch_sig_str_mean,f_clinch_sig_str_last_3_mean,o_clinch_sig_str_last_3_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_distance_sig_str_att_mean,o_distance_sig_str_att_mean,f_distance_sig_str_att_last_3_mean,o_distance_sig_str_att_last_3_mean,f_clinch_sig_str_att_mean,o_clinch_sig_str_att_mean,f_clinch_sig_str_att_last_3_mean,o_clinch_sig_str_att_last_3_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean
7192,1994-03-11,Johnny Rhode,David Levick,W,1,1,9,1,2,1,1,1,15,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7193,1994-03-11,Patrick Smit,Ray Wizar,W,0,1,0,1,0,0,0,1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7194,1994-03-11,Scott Morri,Sean Daughert,W,0,1,0,0,0,0,0,1,0,3,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
for feature in FEATURES:
    data[f'{feature}_dif'] = data[f'f_{feature}'] - data[f'o_{feature}']
    data[f'{feature}_att_dif'] = data[f'f_{feature}_att'] - data[f'o_{feature}_att']
    data[f'{feature}_mean_dif'] = data[f'f_{feature}_mean'] - data[f'o_{feature}_mean']
    data[f'{feature}_att_mean_dif'] = data[f'f_{feature}_att_mean'] - data[f'o_{feature}_att_mean']
    data[f'{feature}_last_{N}_mean'] = data[f'f_{feature}_last_{N}_mean'] - data[f'o_{feature}_last_{N}_mean']

data.head(3)

,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_distance_sig_str_mean,o_distance_sig_str_mean,f_distance_sig_str_last_3_mean,o_distance_sig_str_last_3_mean,f_clinch_sig_str_mean,o_clinch_sig_str_mean,f_clinch_sig_str_last_3_mean,o_clinch_sig_str_last_3_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_distance_sig_str_att_mean,o_distance_sig_str_att_mean,f_distance_sig_str_att_last_3_mean,o_distance_sig_str_att_last_3_mean,f_clinch_sig_str_att_mean,o_clinch_sig_str_att_mean,f_clinch_sig_str_att_last_3_mean,o_clinch_sig_str_att_last_3_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean,distance_sig_str_dif,distance_sig_str_att_dif,distance_sig_str_mean_dif,distance_sig_str_att_mean_dif,distance_sig_str_last_3_mean,clinch_sig_str_dif,clinch_sig_str_att_dif,clinch_sig_str_mean_dif,clinch_sig_str_att_mean_dif,clinch_sig_str_last_3_mean,ground_sig_str_dif,ground_sig_str_att_dif,ground_sig_str_mean_dif,ground_sig_str_att_mean_dif,ground_sig_str_last_3_mean
0,2023-09-23,Rafael Fiziev,Mateusz Gamrot,L,18,0,0,13,1,2,32,0,0,35,1,4,55.375000,30.428571,69.0,41.000000,6.250000,2.857143,6.333333,3.333333,0.500000,3.857143,0.333333,3.333333,111.625000,63.857143,142.333333,79.0,8.375000,3.142857,7.333333,4.000000,0.625000,5.714286,0.666667,4.000000,5,-3,24.946429,47.767857,28.000000,-1,-1,3.392857,5.232143,3.000000,-2,-4,-3.357143,-5.089286,-3.0
1,2023-09-23,Bryce Mitchell,Dan Ige,W,17,1,5,16,2,0,29,3,8,51,3,1,14.571429,40.142857,24.0,48.000000,1.285714,4.000000,0.333333,4.333333,15.142857,7.142857,14.333333,2.333333,31.857143,96.000000,46.333333,112.0,2.000000,6.071429,0.333333,6.000000,18.571429,9.571429,19.333333,3.333333,1,-22,-25.571429,-64.142857,-24.000000,-1,0,-2.714286,-4.071429,-4.000000,5,7,8.000000,9.000000,12.0
2,2023-09-23,Marina Rodrigue,Michelle Waterson-Gomez,W,12,53,5,4,7,2,23,73,6,17,9,6,48.363636,41.000000,29.0,51.333333,15.272727,10.461538,5.333333,5.000000,6.000000,5.923077,7.000000,3.000000,122.818182,104.538462,90.000000,138.0,18.727273,12.769231,7.000000,7.333333,7.363636,7.230769,7.666667,4.333333,8,6,7.363636,18.279720,-22.333333,46,64,4.811189,5.958042,0.333333,3,0,0.076923,0.132867,4.0


In [55]:
X = [f'{feature}_mean_dif' for feature in FEATURES] + [f'{feature}_att_mean_dif' for feature in FEATURES] + [f'{feature}_last_{N}_mean' for feature in FEATURES]
y = 'result'
X

['distance_sig_str_mean_dif',
 'clinch_sig_str_mean_dif',
 'ground_sig_str_mean_dif',
 'distance_sig_str_att_mean_dif',
 'clinch_sig_str_att_mean_dif',
 'ground_sig_str_att_mean_dif',
 'distance_sig_str_last_3_mean',
 'clinch_sig_str_last_3_mean',
 'ground_sig_str_last_3_mean']

In [56]:
set_train_test(X_columns=X, y_columns=y, data=data, test_size=0.3, valid_size=0)
data.tail(3)

Data split: Data=(7195, 56), train_set=(5043, 56), validation_set=(0, 56), test_set=(2152, 56)


,set,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_distance_sig_str_mean,o_distance_sig_str_mean,f_distance_sig_str_last_3_mean,o_distance_sig_str_last_3_mean,f_clinch_sig_str_mean,o_clinch_sig_str_mean,f_clinch_sig_str_last_3_mean,o_clinch_sig_str_last_3_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_distance_sig_str_att_mean,o_distance_sig_str_att_mean,f_distance_sig_str_att_last_3_mean,o_distance_sig_str_att_last_3_mean,f_clinch_sig_str_att_mean,o_clinch_sig_str_att_mean,f_clinch_sig_str_att_last_3_mean,o_clinch_sig_str_att_last_3_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean,distance_sig_str_dif,distance_sig_str_att_dif,distance_sig_str_mean_dif,distance_sig_str_att_mean_dif,distance_sig_str_last_3_mean,clinch_sig_str_dif,clinch_sig_str_att_dif,clinch_sig_str_mean_dif,clinch_sig_str_att_mean_dif,clinch_sig_str_last_3_mean,ground_sig_str_dif,ground_sig_str_att_dif,ground_sig_str_mean_dif,ground_sig_str_att_mean_dif,ground_sig_str_last_3_mean
7192,train,1994-03-11,Johnny Rhode,David Levick,W,1,1,9,1,2,1,1,1,15,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1,0.0,0.0,0.0,-1,-1,0.0,0.0,0.0,8,14,0.0,0.0,0.0
7193,train,1994-03-11,Patrick Smit,Ray Wizar,W,0,1,0,1,0,0,0,1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0.0,0.0,0.0,1,1,0.0,0.0,0.0,0,0,0.0,0.0,0.0
7194,test,1994-03-11,Scott Morri,Sean Daughert,W,0,1,0,0,0,0,0,1,0,3,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-3,0.0,0.0,0.0,1,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0


### Logistic regression model train

In [57]:
# train model
X_train = data.loc[data['set'] == 'train', X].values
y_train = data.loc[data['set'] == 'train', y].values
    
# Create a logistic regression model
model = LogisticRegression(C=0.1)

# Fit the model to the training data
model.fit(X_train, y_train)
model

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1)

In [58]:
# make predictions
data[f'{y}_pred'] = model.predict(data[X])
data[f'{y}_pred_proba'] = [max(item) for item in model.predict_proba(data[X])] #get max probability 
data.tail(3)

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


,set,date,fighter,opponent,result,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_distance_sig_str_att,f_clinch_sig_str_att,f_ground_sig_str_att,o_distance_sig_str_att,o_clinch_sig_str_att,o_ground_sig_str_att,f_distance_sig_str_mean,o_distance_sig_str_mean,f_distance_sig_str_last_3_mean,o_distance_sig_str_last_3_mean,f_clinch_sig_str_mean,o_clinch_sig_str_mean,f_clinch_sig_str_last_3_mean,o_clinch_sig_str_last_3_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_distance_sig_str_att_mean,o_distance_sig_str_att_mean,f_distance_sig_str_att_last_3_mean,o_distance_sig_str_att_last_3_mean,f_clinch_sig_str_att_mean,o_clinch_sig_str_att_mean,f_clinch_sig_str_att_last_3_mean,o_clinch_sig_str_att_last_3_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean,distance_sig_str_dif,distance_sig_str_att_dif,distance_sig_str_mean_dif,distance_sig_str_att_mean_dif,distance_sig_str_last_3_mean,clinch_sig_str_dif,clinch_sig_str_att_dif,clinch_sig_str_mean_dif,clinch_sig_str_att_mean_dif,clinch_sig_str_last_3_mean,ground_sig_str_dif,ground_sig_str_att_dif,ground_sig_str_mean_dif,ground_sig_str_att_mean_dif,ground_sig_str_last_3_mean,result_pred,result_pred_proba
7192,train,1994-03-11,Johnny Rhode,David Levick,W,1,1,9,1,2,1,1,1,15,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1,0.0,0.0,0.0,-1,-1,0.0,0.0,0.0,8,14,0.0,0.0,0.0,W,0.639264
7193,train,1994-03-11,Patrick Smit,Ray Wizar,W,0,1,0,1,0,0,0,1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0.0,0.0,0.0,1,1,0.0,0.0,0.0,0,0,0.0,0.0,0.0,W,0.639264
7194,test,1994-03-11,Scott Morri,Sean Daughert,W,0,1,0,0,0,0,0,1,0,3,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-3,0.0,0.0,0.0,1,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,W,0.639264


In [59]:
# Evaluate model
test_data = data[data.set=='test']
right_pred_test = len(test_data[test_data.result_pred == test_data.result])
all_test = len(test_data)
acc = right_pred_test/all_test
print(f'Accuracy = {acc}')

Accuracy = 0.6473048327137546


In [63]:
# Evaluate high probability predictions only
HIGH_PROBABILITY_SCORE = 0.7
test_data = data[(data.set=='test')&(data.result_pred_proba > HIGH_PROBABILITY_SCORE)]
right_pred_test = len(test_data[test_data.result_pred == test_data.result])
all_test = len(test_data)
acc = right_pred_test/all_test
print(f'Accuracy = {acc}')

Accuracy = 0.7283236994219653


In [64]:
all_test

173